# 🚀 3차원 미사일 방어 시스템 - 정리본

## 문제 정의
- 적 미사일 궤도: P_H1(t) = (250-20t, 300-25t, 400-4.9t²)
- 알파 기지 위치: S_A = (50, 20, 0)
- 요격 미사일 속력: 120 m/s
- 목표: 가장 빠른 요격 시간과 3D 시각화

In [1]:
# 필수 라이브러리
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import matplotlib.font_manager as fm
import platform

# 한글 폰트 설정
def set_korean_font():
    system = platform.system()
    if system == 'Windows':
        korean_fonts = ['Malgun Gothic', 'Arial Unicode MS', 'Gulim']
        for font_name in korean_fonts:
            try:
                font_list = [f.name for f in fm.fontManager.ttflist]
                if font_name in font_list:
                    plt.rcParams['font.family'] = font_name
                    plt.rcParams['axes.unicode_minus'] = False
                    return True
            except:
                continue
    return False

font_success = set_korean_font()
print(f"한글 폰트 설정: {'성공' if font_success else '실패'}")

한글 폰트 설정: 성공


In [2]:
# 기본 설정 및 함수 정의
def enemy_missile_position(t):
    """적 미사일 궤도 P_H1(t)"""
    return np.array([250-20*t, 300-25*t, 400-4.9*t**2])

def enemy_velocity(t):
    """적 미사일 속도"""
    return np.array([-20, -25, -9.8*t])

def enemy_speed(t):
    """적 미사일 속력"""
    return np.linalg.norm(enemy_velocity(t))

# 초기 조건
S_A = np.array([50, 20, 0])  # 알파 기지
v_I = 120  # 요격 미사일 속력

# 적 미사일 초기 속력
initial_speed = enemy_speed(0)
print(f"\n=== 초기 조건 ===")
print(f"알파 기지 위치: {S_A}")
print(f"요격 미사일 속력: {v_I} m/s")
print(f"적 미사일 초기 속력: {initial_speed:.2f} m/s")


=== 초기 조건 ===
알파 기지 위치: [50 20  0]
요격 미사일 속력: 120 m/s
적 미사일 초기 속력: 32.02 m/s


In [3]:
# 요격 시간 계산 (핵심 알고리즘)
def find_intercept_time():
    """가장 빠른 요격 시간 계산"""
    def equation(t):
        if t <= 0:
            return float('inf')
        enemy_pos = enemy_missile_position(t)
        distance = np.linalg.norm(enemy_pos - S_A)
        return distance - v_I * t
    
    # 해 찾기
    t_values = np.linspace(0.1, 10, 1000)
    solutions = []
    
    for i in range(len(t_values)-1):
        if equation(t_values[i]) * equation(t_values[i+1]) < 0:
            # 이분법
            left, right = t_values[i], t_values[i+1]
            for _ in range(50):
                mid = (left + right) / 2
                if equation(mid) * equation(left) < 0:
                    right = mid
                else:
                    left = mid
                if abs(right - left) < 1e-10:
                    break
            solutions.append((left + right) / 2)
    
    return min(solutions) if solutions else None

# 계산 실행
t_impact = find_intercept_time()
P_impact = enemy_missile_position(t_impact)
direction_vector = P_impact - S_A
u = direction_vector / np.linalg.norm(direction_vector)

print(f"\n=== 계산 결과 ===")
print(f"요격 시간: {t_impact:.4f} 초")
print(f"요격 지점: ({P_impact[0]:.1f}, {P_impact[1]:.1f}, {P_impact[2]:.1f})")
print(f"발사 방향: ({u[0]:.4f}, {u[1]:.4f}, {u[2]:.4f})")


=== 계산 결과 ===
요격 시간: 3.4503 초
요격 지점: (181.0, 213.7, 341.7)
발사 방향: (0.3164, 0.4679, 0.8252)


In [4]:
# 3D 시각화
def create_3d_visualization():
    """깔끔한 3D 시각화"""
    # 궤적 계산
    time_points = np.linspace(0, t_impact*1.2, 100)
    enemy_trajectory = np.array([enemy_missile_position(t) for t in time_points])
    
    intercept_times = np.linspace(0, t_impact, 50)
    intercept_trajectory = np.array([S_A + v_I * t * u for t in intercept_times])
    
    # 플롯 생성
    fig = go.Figure()
    
    # 도시
    fig.add_trace(go.Scatter3d(
        x=[0], y=[0], z=[0],
        mode='markers',
        marker=dict(size=15, color='red', symbol='diamond'),
        name='🏙️ 도시'
    ))
    
    # 알파 기지
    fig.add_trace(go.Scatter3d(
        x=[S_A[0]], y=[S_A[1]], z=[S_A[2]],
        mode='markers',
        marker=dict(size=12, color='blue', symbol='square'),
        name='🏭 알파 기지'
    ))
    
    # 적 미사일 궤적
    fig.add_trace(go.Scatter3d(
        x=enemy_trajectory[:, 0],
        y=enemy_trajectory[:, 1],
        z=enemy_trajectory[:, 2],
        mode='lines',
        line=dict(color='red', width=5),
        name='🚀 적 미사일'
    ))
    
    # 요격 미사일 궤적
    fig.add_trace(go.Scatter3d(
        x=intercept_trajectory[:, 0],
        y=intercept_trajectory[:, 1],
        z=intercept_trajectory[:, 2],
        mode='lines',
        line=dict(color='blue', width=5),
        name='🛡️ 요격 미사일'
    ))
    
    # 요격 지점
    fig.add_trace(go.Scatter3d(
        x=[P_impact[0]], y=[P_impact[1]], z=[P_impact[2]],
        mode='markers',
        marker=dict(size=20, color='orange', symbol='x'),
        name='💥 요격 지점'
    ))
    
    # 레이아웃
    fig.update_layout(
        title=dict(
            text='🚀 3차원 미사일 방어 시스템',
            x=0.5,
            font=dict(size=20)
        ),
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.2)),
            aspectmode='cube'
        ),
        width=900,
        height=700
    )
    
    return fig

# 시각화 생성
fig = create_3d_visualization()
fig.show()

print("\n✅ 3D 시각화 완료!")


✅ 3D 시각화 완료!


In [5]:
# 조기 탐지 시스템 분석
def analyze_early_detection(detection_delay):
    """조기 탐지 효과 분석"""
    def predictive_equation(t_flight):
        if t_flight <= 0:
            return float('inf')
        total_time = detection_delay + t_flight
        enemy_pos = enemy_missile_position(total_time)
        distance = np.linalg.norm(enemy_pos - S_A)
        return distance - v_I * t_flight
    
    # 해 찾기
    t_values = np.linspace(0.01, 10, 1000)
    solutions = []
    
    for i in range(len(t_values)-1):
        if predictive_equation(t_values[i]) * predictive_equation(t_values[i+1]) < 0:
            left, right = t_values[i], t_values[i+1]
            for _ in range(50):
                mid = (left + right) / 2
                if predictive_equation(mid) * predictive_equation(left) < 0:
                    right = mid
                else:
                    left = mid
                if abs(right - left) < 1e-10:
                    break
            solutions.append((left + right) / 2)
    
    if solutions:
        t_flight = min(solutions)
        total_time = detection_delay + t_flight
        intercept_point = enemy_missile_position(total_time)
        improvement = ((t_impact - total_time) / t_impact) * 100
        
        return {
            'success': True,
            'total_time': total_time,
            'intercept_point': intercept_point,
            'improvement': improvement
        }
    return {'success': False}

# 다양한 탐지 지연 분석
delays = [0.0, 0.1, 0.2, 0.5, 1.0]
print(f"\n=== 조기 탐지 분석 ===")
print(f"{'탐지지연(초)':<12} {'요격시간(초)':<12} {'요격고도(m)':<12} {'성능개선(%)':<12}")
print("-" * 50)

for delay in delays:
    if delay == 0:
        print(f"{delay:<12.1f} {t_impact:<12.3f} {P_impact[2]:<12.0f} {'기준':<12}")
    else:
        result = analyze_early_detection(delay)
        if result['success']:
            print(f"{delay:<12.1f} {result['total_time']:<12.3f} {result['intercept_point'][2]:<12.0f} {result['improvement']:<12.1f}")
        else:
            print(f"{delay:<12.1f} {'실패':<12} {'N/A':<12} {'N/A':<12}")

print("\n✅ 조기 탐지 분석 완료!")


=== 조기 탐지 분석 ===
탐지지연(초)      요격시간(초)      요격고도(m)      성능개선(%)     
--------------------------------------------------
0.0          3.450        342          기준          
0.1          3.523        339          -2.1        
0.2          3.594        337          -4.2        
0.5          3.809        329          -10.4       
1.0          4.161        315          -20.6       

✅ 조기 탐지 분석 완료!


# 📊 결론

## 핵심 결과
1. **기본 요격**: 가장 빠른 요격이 가능한 시간과 지점 계산
2. **3D 시각화**: 미사일 궤적과 요격 과정을 직관적으로 표현
3. **조기 탐지**: 탐지 지연이 요격 성능에 미치는 영향 분석

## 주요 발견
- 적 미사일 초기 속력: **32.02 m/s**
- 조기 탐지가 성능 향상에 결정적 역할
- 탐지 지연 시간 최소화의 중요성